In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy
import math
import torch
import sys
sys.path.insert(0,'C:\\Users\\twilkeni\\AppData\\Local\\anaconda3\\envs\\pytorch-env\\site-packages')
import gpytorch
# import pyro
# from pyro.infer.mcmc import NUTS, MCMC, HMC
from copy import deepcopy
from functools import partial
from scipy.stats import multivariate_normal
from qwiic_ads1115.qwiic_ads1115 import QwiicAds1115
import time


In [9]:
ads = QwiicAds1115()
ads.os = '0' # no effect
ads.mux = '100' # Analog In, channel 0, compare to GND
ads.gain = '000' # +/- 6.144V
ads.mode = '0' # continuous
ads.data_rate = '100' # 128 SPS
ads.comp_que = '11' # disable comparator
ads.scale = 1 # 1V = 1kPa delta between ports
if ads.is_connected():
    ads.configure()
    time.sleep(0.5)
    ads.calibrate()
    
    # Confirm calibration set:
    if abs(ads.get_measurement()) > 0.1:
        print("Pressure calibration has likely failed")
    else:
        print("ADS1115 successfully initialized and calibrated")
else:
    print("ADS1115 not connected")

ADS1115 successfully initialized and calibrated


In [13]:
def velocity(pressure_diff,rho=1.204):
    """VELOCITY calculates the velocity of the airflow given a pitot
    pressure difference measurement. Calculates velocity regardless
    of whether the pitot tube is setup for positive voltage output
    on the ADC, or negative voltage output (based on how the pressure
    tubes are connected to the pressure sensor).

    Args:
        pressure_diff (float): total pressure - static pressure [kPa]
            (output of MPXV7002 is 1V/kPa)
        rho (float): density of the fluid [kg/m^3]
            (1.204 kg/m3 at sea level per International Standard
            Atmosphere)
    Returns:
        vel (float): calculated velocity [m/s]
    """
    # calculate velocity per the following resource:
    # https://www.grc.nasa.gov/www/k-12/VirtualAero/BottleRocket/airplane/pitot.html
    vel = np.sqrt(2*abs(pressure_diff) / rho)
    return vel

In [14]:
velocity(ads.get_measurement())

0.10384029713444146

In [15]:
ads.get_measurement()

-0.004053749999999745